In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with TEASER-S model and simulated feedback

In [2]:
from pathlib import Path
import numpy as np

import teaser.util as util
import teaser.evaluation as evaluation

from teaser.algorithm.teaser_s import TEASER_S

## Datasets

In [3]:
## Uncomment corresponding directory.
# DATA_DIR = Path('../../data/ML20M/')
# DATA_DIR = Path('../../data/Amazon Video Games/')
DATA_DIR = Path('../../data/Publiq/')

In [4]:
# Shouldn't need to change this info
INTERACTIONS_TRAIN_PATH = DATA_DIR / 'interactions.train.csv'
INTERACTIONS_VALIN_PATH = DATA_DIR / 'interactions.valin.csv'
INTERACTIONS_VALOUT_PATH = DATA_DIR / 'interactions.valout.csv'
INTERACTIONS_TESTIN_PATH = DATA_DIR / 'interactions.testin.csv'
INTERACTIONS_TESTOUT_PATH = DATA_DIR / 'interactions.testout.csv'

METADATA_PATH = DATA_DIR / 'features.csv'

ITEM_ID = 'itemId'
USER_ID = 'userId'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = TEASER_S

# ML20M
# L2_1 = 100
# L2_2 = 100000000
# RHO = 1000
# DELTA = 0
# MAX_ITER = 10

# AVG
# L2_1 = 500
# L2_2 = 2000
# RHO = 10000
# DELTA = 0
# MAX_ITER = 15

# Publiq
L2_1 = 500
L2_2 = 50000
RHO = 1000
DELTA = 0
MAX_ITER = 10

HYPERPARAMS = {'l2_1': L2_1, 'l2_2': L2_2, 'rho': RHO, 'delta': DELTA,'max_iterations': MAX_ITER}
HYPERPARAMS

{'l2_1': 500, 'l2_2': 50000, 'rho': 1000, 'delta': 0, 'max_iterations': 10}

## Parse data

In [6]:
S, tags = util.parse_metadata(METADATA_PATH, item_id=ITEM_ID)
n, t = S.shape  # amount of items and tags

X = util.parse_interactions(INTERACTIONS_TRAIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

Xtest_in = util.parse_interactions(INTERACTIONS_TESTIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)
Xtest_out = util.parse_interactions(INTERACTIONS_TESTOUT_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

In [7]:
# # debug
# d_items = 1000
# S = S[:d_items,:]
# tags = tags[:d_items]
# X = X[:,:d_items]
# Xtest_in = Xtest_in[:,:d_items]
# Xtest_out = Xtest_out[:,:d_items]

## Evaluate model with optimal hyperparams on test set

In [8]:
# Precomput XTX and decomposition of XTX, since it's needed for all hyperparameters combinations and doesn't change
XTX = (X.T @ X).toarray()
p, U = np.linalg.eigh(XTX)
fit_params = {'XTX': XTX, 'p': p, 'U': U}

In [9]:
%%time
alg = ALG(**HYPERPARAMS)
alg.fit(X, S, **fit_params)

Decompose DTD


norm E 0.06109046846017588
diag norm: 0.13716239344731093
diag_diff: 0.3653087697280152

norm E 0.05745460125774677
diag norm: 0.12476903512529049
diag_diff: 0.2159972038968077
rho * change E: 19.149511549151462

norm E 0.05524703366562962
diag norm: 0.1160299699011726
diag_diff: 0.15143179116814817
rho * change E: 3.3612506720066424

norm E 0.05332817588331851
diag norm: 0.10823759225144375
diag_diff: 0.11805019479217495
rho * change E: 2.963323876390799

norm E 0.05167057224223128
diag norm: 0.1013656802857356
diag_diff: 0.09947053954134188
rho * change E: 2.6851739263441474

norm E 0.050259392227827986
diag norm: 0.09544088107745421
diag_diff: 0.08791502707116965
rho * change E: 2.410034541172794

norm E 0.04905430113792256
diag norm: 0.09034126728937328
diag_diff: 0.07953870382847783
rho * change E: 2.1798077068468156

norm E 0.048024641432948746
diag norm: 0.08595740020083947
diag_diff: 0.07243282767432786
rho * change E: 1.9617776254192858

norm E 0.04716286002496448
diag norm: 0

In [10]:
print("Performance on test set with simulation")
for POS_FEEDBACK in [1, 2]:
    for STRENGTH in [1, 3]:
        print("Pos feedback count:", POS_FEEDBACK, "with strength (out of 5):", STRENGTH)
        score = evaluation.simulate(alg, Xtest_in, Xtest_out, S, pos_feedback=POS_FEEDBACK, strength=STRENGTH, repeats=3)
    print()

Performance on test set with simulation
Pos feedback count: 1 with strength (out of 5): 1
Evaluating with 1500 users
Average Recall@20 0.402
Average Recall@100 0.606
Average nDCG@100 0.268
Pos feedback count: 1 with strength (out of 5): 3
Evaluating with 1500 users
Average Recall@20 0.475
Average Recall@100 0.715
Average nDCG@100 0.328

Pos feedback count: 2 with strength (out of 5): 1
Evaluating with 1500 users
Average Recall@20 0.443
Average Recall@100 0.672
Average nDCG@100 0.305
Pos feedback count: 2 with strength (out of 5): 3
Evaluating with 1500 users
Average Recall@20 0.625
Average Recall@100 0.858
Average nDCG@100 0.423

